In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import math 
import collections
from pycocotools.coco import COCO
import requests
import plotly.express as px
import plotly.graph_objects as go
from os import listdir
from os.path import isfile, join
import base64
import itertools
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
sns.set(style='whitegrid', font_scale=1.6, font='Georgia', context='paper')

from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import operator
import plotly.figure_factory as ff
import scipy
import pickle 
from sklearn.neighbors import KernelDensity
from tqdm import tqdm
from collections import OrderedDict

In [ ]:
base_path = '../../../snap/vqa/lxr111_multilabel_full_run_3/'
sampling_ids = '../../../src/dataset_selection/sampling/samples/LXR111/multilabel_full/beta/beta_kernel/tophat/seed_965/alpha_2_beta_1_budget_10.pkl'
df = pd.read_pickle(base_path+"datamap_metrics.pkl")

with open(sampled_question_ids_path, 'rb') as f:
    sampled_ids = pickle.load(f)

df = dataframe.loc[dataframe['question_id'].isin(sampled_ids)]


region = 'ambiguous'
confidence_threshold = 0.01
variability_threshold = 0.01

# stacked bar chart -- fig 6 across all samples
if region == 'hard':
    # segment instances thresholded on confidence and variability to separate regions on datamap 
    df = df.loc[(df['confidence'] < confidence_threshold) & (df['variability'] < variability_threshold)]
elif region == 'easy':
    df = df.loc[(df['confidence'] > confidence_threshold) & (df['variability'] < variability_threshold)]
else:
    df = df.loc[(df['confidence'] > 0.4) &(df['confidence'] < 0.8) & (df['variability'] > 0.4)]



In [ ]:
# bin by variability
weight_counts = {
    "0-0.1": np.array([70, 31, 58]),
    "0.1-0.2": np.array([82, 37, 66]),
    "0.2-0.3": np.array([82, 37, 66]),
    "0.3-0.4": np.array([82, 37, 66]),
    "0.4-0.5": np.array([82, 37, 66]),
}
# bin by confidence
weight_counts = {
    "0-0.2": np.array([70, 31, 58]),
    "0.2-0.4": np.array([82, 37, 66]),
    "0.4-0.6": np.array([82, 37, 66]),
    "0.6-0.8": np.array([82, 37, 66]),
    "0.8-1.0": np.array([82, 37, 66]),
}
# bin by region
weight_counts = {
    "easy": np.array([70, 31, 58]),
    "ambiguous": np.array([82, 37, 66]),
    "hard": np.array([82, 37, 66]),
}


In [ ]:
# for random sampling and beta sampling, get top question types like you did in test.ipynb and plot their binned confidence scores and binned variability scores


# also calculate skew metrics comparing random and beta sampling class distributions 